In [ ]:
import pandas as pd
import csv
import numpy as np
import mplcursors
import matplotlib.pyplot as plt

In [ ]:
### Make matrix with all movie
all_tropes = []
matrix_maker = []
genres = []
with open("C:/Users/jbro0/OneDrive/Desktop/College_Stuff/PoCSVerse/Project/Github Stuff/Pocs_Film_Project/Data/liteweight/2020_genre_counts_by_trope.csv", "r") as file:
    reader = csv.reader(file)
    row1 = next(reader)
    for i in range(2,29):
            genres.append(row1[i])
    for line in reader:
        #line_list = line.split(",")
        all_tropes.append(line[1])
        genre_counts = []
        for i in range(2,29):
            genre_counts.append(int(line[i]))
        normalization = sum(genre_counts)
        try:
            genre_percents = [genre_count / normalization for genre_count in genre_counts]
            matrix_maker.append(genre_percents)
        except ZeroDivisionError:
            matrix_maker.append(genre_counts)
    
bad_matrix = np.array(matrix_maker)
matrix = bad_matrix.transpose()

In [ ]:
### Define step function
def H(x,t):
    if x < t:
        return 0
    elif x >= t:
        return 1

In [ ]:
### Fill in movie vector
def vec_filler(x):
    movie_vec = [0] * len(all_tropes)
    with open("C:/Users/jbro0/Downloads/Alien Tropes - CodeTest.tsv", "r") as file:
        tsv_reader = csv.reader(file, delimiter='\t')
        next(tsv_reader)
        for line in tsv_reader:
            trope = line[0].replace(" ", "")
            if trope in all_tropes:
                vec_index = all_tropes.index(trope)
                if line[1] == "Yes" or line[2] == "Yes" and line[5] != "Yes":
                    movie_vec[vec_index] -= H(x, float(line[6]))
                elif line[5] == "Yes":
                    pass
                else:
                    movie_vec[vec_index] += H(x, float(line[6]))
            elif trope not in all_tropes:
                pass ### Need to find a way to keep track of this
    normalizer = sum(movie_vec)
    movie_vec = [element / normalizer for element in movie_vec]
    movie_vec = np.array(movie_vec)
    return movie_vec

In [ ]:
x = 0
snapshots = []
x_points = []
while x < 7014:
    new_basis_vec = np.matmul(matrix, vec_filler(x))
    new_basis_vec = list(new_basis_vec)
    snapshots.append(new_basis_vec)
    x_points.append(x)
    x += 1
    print(x)

In [ ]:
genre_proportions = np.transpose(snapshots)
genre = "Horror"  ### Input genre here

_index = genres.index(genre)
props = list(genre_proportions[_index])

### This is the genre makeup data

In [ ]:
### Get titles, times, and descriptions of tropes

labels = []
times = []
with open("C:/Users/jbro0/Downloads/Alien Tropes - CodeTest.tsv", "r") as file:
    tsv_reader = csv.reader(file, delimiter='\t')
    next(tsv_reader)
    for line in tsv_reader:
        trope = line[0].replace(" ", "")
        inverted_or_defied = line[1]
        averted_or_subverted = line[2]
        descriptor = line[3]
        background = line[4]
        setup = line[5]
        start_time = float(line[6])
        end_time = float(line[7])
        if background == "Yes":
            continue
        elif inverted_or_defied == "Yes":
            label = "Inverted/Defied " + trope + ": " + descriptor
            labels.append(label)
            times.append(start_time)
        elif averted_or_subverted == "Yes":
            label = "Averted/Subverted " + trope + ": " + descriptor
            labels.append(label)
            times.append(start_time)
        elif setup == "Yes":
            label_one = "Setup - " + trope + ": " + descriptor
            label_two = "Conclusion - " + trope + ": " + descriptor
            labels.append(label_one)
            times.append(start_time)
            labels.append(label_two)
            times.append(end_time)
        else:
            label = trope + ": " + descriptor
            labels.append(label)
            times.append(start_time)


In [ ]:
times = list(set(times))
labels = [""] * len(times) 
with open("C:/Users/jbro0/Downloads/Alien Tropes - CodeTest.tsv", "r") as file:
    tsv_reader = csv.reader(file, delimiter='\t')
    next(tsv_reader)
    for line in tsv_reader:
        trope = line[0].replace(" ", "")
        inverted_or_defied = line[1]
        averted_or_subverted = line[2]
        descriptor = line[3]
        background = line[4]
        setup = line[5]
        start_time = float(line[6])
        end_time = float(line[7])
        if background == "Yes":
            continue
        elif inverted_or_defied == "Yes":
            label = "Inverted/Defied " + trope + ": " + descriptor
            index = times.index(start_time)
            labels[index] += label + "\n"
        elif averted_or_subverted == "Yes":
            label = "Averted/Subverted " + trope + ": " + descriptor
            index = times.index(start_time)
            labels[index] += label + "\n"
        elif setup == "Yes":
            label_one = "Setup - " + trope + ": " + descriptor
            label_two = "Conclusion - " + trope + ": " + descriptor
            index_one = times.index(start_time)
            labels[index_one] += label_one + "\n"
            index_two = times.index(end_time)
            labels[index_two] += label_two + "\n"
        else:
            label = trope + ": " + descriptor
            index = times.index(start_time)
            labels[index] += label + "\n"

In [ ]:
interactable_y = []
for time in times:
    indecks = x_points.index(int(time))
    interactable_y.append(props[indecks])

In [ ]:
### This is the problematic block, the code should be fine but it's not popping up in a window like it should

%matplotlib widget
fig, ax = plt.subplots(figsize =(6,6))
plt.title(genre + " make up of Alien, with tropes")
plt.xlabel("Elapsed Time(s)")
plt.ylabel("% Makeup")
ax.plot(x_points, props, c = "blue")
ax.scatter(times, interactable_y, c = "darkorange", s = 60)
mplcursors.cursor(ax, hover = True).connect("add", lambda sel: sel.annotation.set_text(labels[sel.index]))
plt.show()